Install Required Packages:

In [53]:
!pip install llama-index
!pip install PyPDF2
!pip install openai



Read and Process DocumentsImport Libraries and Set Up API Key

In [54]:
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext,load_index_from_storage # Import StorageContext
import PyPDF2
import openai
import os
os.environ["OPENAI_API_KEY"] = 'openai.api_key'

openai.api_key = os.getenv("OPENAI_API_KEY")



Read and Index the PDF Document

In [55]:
def read_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

# Assuming you have uploaded the PDF file to your Colab environment
pdf_path = '/content/sap-s4hana-mm-configuration-document.pdf'
document_text = read_pdf(pdf_path)

# Create a Document object from the text
document = Document(text=document_text, metadata={"title": "SAP S/4HANA Materials Management"})

# Create the index using the new import structure
index = VectorStoreIndex.from_documents(documents=[document])

# Persist the index using the storage context
index.storage_context.persist(persist_dir="./index")


Query the Index and Generate Responses

In [67]:

def load_index(index_path):
    # Load the index from storage context
    storage_context = StorageContext.from_defaults(persist_dir=index_path)
    return load_index_from_storage(storage_context)

def query_index(query, index):
    # Use the correct method to perform the query
    response = index.as_query_engine().query(query)
    # Ensure the response is properly accessed
    return response

def generate_response(query, index):
    # Retrieve relevant sections using the index
    response = query_index(query, index)
    context = response.response  # Extract the context from the index query
    prompt = (
        f"{query} Context: {context}\n\n"
        "Please provide a detailed step-by-step guide for configuring material management "
        "in SAP S/4HANA. Include information on defining the enterprise structure, "
        "creating master data, setting up procurement processes, and configuring "
        "inventory management. Explain each step thoroughly with specific transaction "
        "codes and paths in the SAP system."
    )

# Generate a response using OpenAI's API
    client = openai.OpenAI(api_key=openai.api_key)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an SAP S/4HANA expert."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        n=1,
        stop=None,
        temperature=0.7,
    )
    return response.choices[0].message.content.strip()

# Load the index
index = load_index(index_path)

# Example usage
query = "How can we configure material management in SAP S/4HANA?"

response = generate_response(query, index)
print("Response:", response)


Response: Configuring Material Management (MM) in SAP S/4HANA involves several steps, including defining the enterprise structure, creating master data, setting up procurement processes, and configuring inventory management. Below is a detailed step-by-step guide to help you through the process.

### Defining the Enterprise Structure

1. **Define Company Code:**
   - Transaction Code: `OX02`
   - Path: `SPRO` > `Enterprise Structure` > `Definition` > `Financial Accounting` > `Define Company`

2. **Define Plant:**
   - Transaction Code: `OX10`
   - Path: `SPRO` > `Enterprise Structure` > `Definition` > `Logistics - General` > `Define, Copy, Delete, Check Plant`

3. **Define Storage Location:**
   - Transaction Code: `OX09`
   - Path: `SPRO` > `Enterprise Structure` > `Definition` > `Materials Management` > `Maintain Storage Location`

4. **Define Purchasing Organization:**
   - Transaction Code: `OX08`
   - Path: `SPRO` > `Enterprise Structure` > `Definition` > `Materials Management` > 